In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin.gz', binary=True)
model_n = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin.gz', binary=True) 
model_n.init_sims(replace=True) #normalized

**Read data from google drive:**

In [0]:
qna = pd.read_csv('/content/drive/My Drive/Colab Notebooks/qna.csv', sep=',',header=None, usecols=[0])
fb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/singtel_fb_questions.csv', sep=',', usecols=[0])
qna.columns = ['question']

**Obtain index of the final question in Q and A as well as in the Facebook posts that we scraped:**

In [0]:
last_qna = qna.tail(1).index.item()
last_fb = fb.tail(1).index.item()

**Test it out with a fake user question (question1) and a QnA question (question2):**

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
question1 = 'Will i be charged if I upgrade to Singtel Fibre Broadband?'
question2 = 'Will there be any charges for conversion if I plan to migrate from SingNet 56K dial-up or Magix plan to Singtel Fibre Broadband service plans?'

question1 = question1.lower().split()
question2 = question2.lower().split()

question1 = [w for w in question1 if w not in stopwords.words('english')]
question2 = [w for w in question2 if w not in stopwords.words('english')]

distance = model.wmdistance(question1, question2)
print('distance = %.4f' % distance)

distance = 3.5535


In [8]:
distance = model_n.wmdistance(question1, question2)
print('normalized distance = %.4f' % distance)

normalized distance = 1.2207


**Now with our own:**

In [14]:
n = 0 #numerator +=1 if we determine that the fb question can be answered by one of the faq question
d = 0 #denominator +=1 every iteration

for i in range(0,last_fb+1):
  question1 = fb.post_text[i]
  
  if isinstance(question1,str):
    
    if '?' in question1:
      question1 = question1.lower().split()
      question1 = [w for w in question1 if w not in stopwords.words('english')]

      for j in range(0,last_qna+1):
        question2 = qna.question[j]
        question2 = question2.lower().split()
        question2 = [w for w in question2 if w not in stopwords.words('english')]

        distance = model.wmdistance(question1, question2)
        distance_n = model_n.wmdistance(question1, question2)

        if (abs(distance-distance_n) > 3):
          n += 1
          break
      
      d += 1
  
print(n/d)

0.6666666666666666
